<a href="https://colab.research.google.com/github/BillyTevin11/Coding-Projects/blob/main/The_MV_Wakashio_Oil_Spillage_in_Mauritius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

An oil spill can be defined as the release of oil into the environment (Lawson,2012). When oil is spilled, it can be very detrimental to the physical, biological and social environment. Some of the effects of oil spillages are fire outbreaks when the oil comes into contact with ignitions that may claim human and animal lives, choking of plants by cutting off oxygen and preventing water absorption by the soils that make them infertile.

The persistency of oil spills in food chains due to lead components has resulted to long term health effects to both humans and animal lives (Ogumor,2013).

There are two types of oil spills:
  1. Marine Oil Spills - Occur on Sea
  2. Terrestrial Oil Spills - Occur on Land (Overholt,2016)

# **Case Study**

Mauritius is known to most people as a popular holiday destination because of its gorgeous beaches, volcanic rocks, its fragile ecosystems of mangrove forests, coral atolls, underwater coral reefs, seagrass, and historic forts. But in early 2020, the tourism industry – a key sector in the Mauritian economy – was hit hard due to the outbreak of COVID-19, hurting the livelihoods of a large part of the local population. And what happened several months later completely devastated the island nation.

On July 25, the Japanese vessel MV Wakashio, transiting too close to Mauritius, crashed into one of its most fragile coral reefs. On August 6, it began leaking fuel, which turned out to be a new type of highly toxic heavy oil. Before the leak was stopped, about 1,000 tonnes of oil had seeped into the Indian Ocean, severely contaminating Mauritius’s shoreline and lagoons in what would become the worst environmental disaster that the country had ever known.

Tens of thousands of people who volunteered for the clean-up, including members of local fishing communities, were exposed to the toxic components of the oil spill, which damaged the island’s food system and degraded its attractiveness for tourism. Three sailors died and one went missing while towing a barge carrying oil from the wreck. More than 50 whales and dolphins washed up dead on the island’s shores; the government is yet to release the full details of the autopsies of the animals which would confirm whether their deaths were caused by the oil spill.

In the months since the tragedy struck Mauritius, instead of ensuring full transparency over the fallout of the oil spill and taking urgent steps to ensure it does not repeat, the government, the Japanese corporation, and international organisations that got involved have withheld crucial information from the public.

Today, uncertainty about Mauritius recovery persists, while no major action has been taken to protect the island’s waters from a future incident.



# Code Execution

In [3]:
# Import the Earth Engine library
import ee

# Import the geemap Python library
import geemap

# Import the folium library
import folium

In [4]:
# Initialize the Earth Engine module
ee.Authenticate()

ee.Initialize(project='mauritius-oil2020')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [5]:
# Define the region of interest around Pointe d’Esny, Mauritius
roi = ee.Geometry.Point([57.74463, -20.43812]).buffer(5000)
Map = geemap.Map()
Map.centerObject(roi, 13)
Map.addLayer(roi, {'color': 'red'}, 'ROI')
Map

Map(center=[-20.438114870556834, 57.74463015180239], controls=(WidgetControl(options=['position', 'transparent…

In [6]:
# Define the date range to focus on the oil spill timeline
start_date = '2020-08-10'
end_date = '2020-08-15'

VV polarization is selected for analyzing Sentinel-1 SAR (Synthetic Aperture Radar) data over water surfaces, which is essential for detecting oil spills.

Polarization Types in Sentinel-1: Sentinel-1 SAR data typically provides two polarization options:

i. VV (Vertical Transmit, Vertical Receive): Both the radar wave transmission and reception are vertically polarized.

ii. VH (Vertical Transmit, Horizontal Receive): Vertical transmission with horizontal reception, often used to enhance surface detail in vegetation and land monitoring.

VV Polarization for Water Surfaces:
1. Surface Smoothness Detection: Water surfaces without oil generally scatter radar signals more uniformly, which returns a low backscatter signal. Oil smooths the water surface further, creating a darker, low-backscatter signal in SAR imagery. VV polarization is particularly sensitive to this effect, making it ideal for distinguishing between clean and oil-covered water.

2. Contrast Enhancement: In VV polarization, oil slicks appear as dark areas against the lighter background of clean water. This enhances the contrast, making it easier to delineate the extent and location of the oil spill. This effect is less pronounced in VH polarization, where backscatter from rough surfaces, such as those in land-based or vegetative areas, may dominate and reduce visibility of oil on water.

Environmental Considerations: VV polarization is less sensitive to certain environmental factors like waves and wind that can increase noise in VH polarization, providing clearer and more consistent results over water bodies. This consistency is especially valuable when monitoring spills over time, as it minimizes variations due to changing sea states.

By selecting VV polarization in the Sentinel-1 imagery, we maximize the visibility of the oil spill on the water surface, ensuring that the SAR data highlights slicks effectively against the surrounding water for accurate mapping and monitoring.

In [7]:
# Load Sentinel-1 SAR GRD imagery, filter by date, location, and polarization (VV is typically better for water surfaces)
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(roi)
             .filterDate(start_date, end_date)
             .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide swath mode
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
             .select('VV'))

In [8]:
print('Number of images in the collection:', sentinel1.size().getInfo())

Number of images in the collection: 1


In [9]:
# The date sentinel1 was obtained or captured by the sensor

print(sentinel1.first().date().format('YYYY-MM-dd').getInfo())

2020-08-10


In [10]:
# Apply a median composite to reduce noise and highlight the spill
composite = sentinel1.median()

**Minimum: -25 and Maximum: 0:**

These values set the range for the backscatter intensity in decibels (dB), which SAR data uses to represent reflectivity.

Lower backscatter values appear darker, while higher values are brighter.

Since oil slicks tend to have lower backscatter values (closer to -25 dB), they will appear dark on the map, contrasting against brighter, high-backscatter areas.

The range from -25 to 0 is chosen because oil-covered water generally falls within this lower intensity range, allowing the slicks to stand out.

To display the oil spill in conventional black-and-white (grayscale), with darker shades representing the oil-covered areas and lighter shades representing clean water, black and white colors are used for visualization.

In this setup, oil slicks will appear in darker shades, making them distinct from other areas.

In [11]:
# Visualization parameters to enhance the appearance of oil slicks
viz_params = {
    'min': -25,        # dB scale minimum for SAR imagery
    'max': 0,          # dB scale maximum
    'palette': ['black', 'white']  # Black to white color palette for contrast
}

In [12]:
# Add composite image to the map with visualization parameters
Map = geemap.Map()
Map.centerObject(roi, 13)
Map.addLayer(composite.clip(roi), viz_params, 'Mauritius Oil Spill on August 10th, 2020')

# Display the map
Map

Map(center=[-20.438114870556834, 57.74463015180239], controls=(WidgetControl(options=['position', 'transparent…

The areas with intensely darker shades are areas that potentially had oil spills. Note, the image is as per the radar image captured on the 10th of August, 2020. Compare the mapped outputs to the NASA report on the reported Oil Spills in the Indian Ocean in the same location around Pointe d’Esny, Mauritius.